In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk
import csv

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
Data = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
#train.drop(columns=['Name','Ticket','Cabin'])  ---> Pourquoi ça ne marche pas ?
#test.drop(columns=['Name','Ticket','Cabin'])
Data.pop('Name')
Data.pop('Ticket')
Data.pop('Cabin')
test.pop('Name')
test.pop('Ticket')
test.pop('Cabin')

train = Data[:int(len(Data)*0.7)].copy()
cross = Data[int(len(Data)*0.7):].copy()

#combine = [train, test]


In [ ]:
#RECHERCHE DES COLONNES INCOMPLETES
test_incompletes=[col for col in test.columns if test[col].isnull().any()]
train_incompletes=[col for col in train.columns if train[col].isnull().any()]
cross_incompletes=[col for col in cross.columns if cross[col].isnull().any()]

train_incompletes, test_incompletes, cross_incompletes


In [ ]:
#Conversion des datas non-numériques

train['Sex'] = train['Sex'].apply({'male':0, 'female':1}.get)
cross['Sex'] = cross['Sex'].apply({'male':0, 'female':1}.get)
test['Sex'] = test['Sex'].apply({'male':0, 'female':1}.get)
train['Embarked'] = train['Embarked'].apply({'C':0, 'S':1, 'Q':0.5}.get)
cross['Embarked'] = cross['Embarked'].apply({'C':0, 'S':1, 'Q':0.5}.get)
test['Embarked'] = test['Embarked'].apply({'C':0, 'S':1, 'Q':0.5}.get)



In [ ]:
#ETUDE DE L'INFLUENCE DU LIEU D'EMBARQUEMENT

#Passagers embarqués à S
S=train.loc[(train['Embarked']==1),:]
#dont survivants
SV=S.loc[(S['Survived']==1),:]
# taux de survie
taux_Embarked_S=SV.Embarked.count()/S.Embarked.count()

#Passagers embarqués à C
C=train.loc[(train['Embarked']==0),:]
#dont survivants
CV=C.loc[(C['Survived']==1),:]
# taux de survie
taux_Embarked_C=CV.Embarked.count()/C.Embarked.count()

print('taux de survie des passagers de S =',taux_Embarked_S,' ; taux de survie des passagers de S =',taux_Embarked_C)






# Proportion des passagers dans chaque classe, selon leur ville d'embarquement

#Passagers embarqués à S
#S=train.loc[(train['Embarked']=="S"),['Survived','Embarked']]
#dont 1ère classe
S1=S.loc[(S['Pclass']==1),:]
#dont 2ème classe
S2=S.loc[(S['Pclass']==2),:]
#dont 3ème classe
S3=S.loc[(S['Pclass']==3),:]
# proportion des passagers de S dans chaque classe :
prop_S_1=S1.Embarked.count()/S.Embarked.count()
prop_S_2=S2.Embarked.count()/S.Embarked.count()
prop_S_3=S3.Embarked.count()/S.Embarked.count()

#Passagers embarqués à C
#S=train.loc[(train['Embarked']=="C"),['Survived','Embarked']]
#dont 1ère classe
C1=C.loc[(C['Pclass']==1),:]
#dont 2ème classe
C2=C.loc[(C['Pclass']==2),:]
#dont 3ème classe
C3=C.loc[(C['Pclass']==3),:]
# proportion des passagers de S dans chaque classe :
prop_C_1=C1.Embarked.count()/C.Embarked.count()
prop_C_2=C2.Embarked.count()/C.Embarked.count()
prop_C_3=C3.Embarked.count()/C.Embarked.count()

print('proportion de passagers de S en 1ère =',prop_S_1,' ; proportion de passagers de S en 2ème =',prop_S_2,' ; proportion de passagers de S en 3ème =', prop_S_3)
print('proportion de passagers de C en 1ère =',prop_C_1,' ; proportion de passagers de C en 2ème =',prop_C_2,' ; proportion de passagers de C en 3ème =', prop_C_3)




# A classe égale, il y a t-il de différence de taux de survie selon le lieu d'embarquement ?

S1S=S1.loc[(S['Survived']==1),:] #Embarqué à S, classe 1, Survivant
S2S=S2.loc[(S['Survived']==1),:] #Embarqué à S, classe 2, Survivant
S3S=S3.loc[(S['Survived']==1),:] #Embarqué à S, classe 3, Survivant
C1S=C1.loc[(C['Survived']==1),:] #Embarqué à C, classe 1, Survivant
C2S=C2.loc[(C['Survived']==1),:] #Embarqué à C, classe 2, Survivant
C3S=C3.loc[(C['Survived']==1),:] #Embarqué à C, classe 3, Survivant

taux_S1=S1S.Embarked.count()/S1.Embarked.count()
taux_S2=S2S.Embarked.count()/S2.Embarked.count()
taux_S3=S3S.Embarked.count()/S3.Embarked.count()
taux_C1=C1S.Embarked.count()/C1.Embarked.count()
taux_C2=C2S.Embarked.count()/C2.Embarked.count()
taux_C3=C3S.Embarked.count()/C3.Embarked.count()

print('taux de survie des passagers de S en 1ère =',taux_S1,' ; taux de survie des passagers de S en 2ème =',taux_S2,' ; taux de survie des passagers de S en 3ème =', taux_S3)
print('taux de survie des passagers de C en 1ère =',taux_C1,' ; taux de survie des passagers de C en 2ème =',taux_C2,' ; taux de survie des passagers de C en 3ème =', taux_C3)

In [ ]:
# CROSSTABS & PLOTS

print(pd.crosstab(train['Survived'],train['SibSp']))
print(pd.crosstab(train['Survived'],train['Parch']))
print(pd.crosstab(train['Survived'],train['Embarked']))
print(pd.crosstab(train['Pclass'],train['Embarked']))

#Plots
surv=train['Survived']
surv = surv.apply({0:'g', 1:'b'}.get)

train.plot('Pclass','Age', c=surv)

In [ ]:
def donnees_suppr() :

# DONNEES SANS LES DONNEES INCOMPLETES
    train_suppr = train.copy()
    train_suppr.dropna(inplace=True)

# NORMALISATION DES DONNEES
    y_suppr=train_suppr['Survived'].copy()
    Id_X = train_suppr['PassengerId'].copy()
    X_suppr=train_suppr.drop(columns = ['Survived', 'PassengerId'])

    X_suppr = (X_suppr - X_suppr.mean()) / (X_suppr.max() - X_suppr.min())

# PREPARATION DES DONNEES TEST
    test_suppr = test.copy()
    test_suppr.dropna(inplace=True)
    
    Id_test = test_suppr['PassengerId'].copy()
    test_suppr=test_suppr.drop(columns = 'PassengerId')

# NORMALISATION DES DONNEES TEST
    test_suppr = (test_suppr - test_suppr.mean()) / (test_suppr.max() - test_suppr.min())


    return X_suppr , test_suppr, y_suppr, Id_test



In [ ]:
##TEST DONNES_SUPPR

# DONNEES SANS LES DONNEES INCOMPLETES
train_suppr = train.copy()
train_suppr.dropna(inplace=True)

# NORMALISATION DES DONNEES

y_suppr=train_suppr['Survived'].copy()
Id_X = train_suppr['PassengerId'].copy()
X_suppr=train_suppr.drop(columns = ['Survived', 'PassengerId'])

X_suppr = (X_suppr - X_suppr.mean()) / (X_suppr.max() - X_suppr.min())

# PREPARATION DES DONNEES TEST
test_suppr = test.copy()
test_suppr.dropna(inplace=True)
    
Id_test = test_suppr['PassengerId'].copy()
test_suppr=test_suppr.drop(columns = 'PassengerId')

# NORMALISATION DES DONNEES TEST
test_suppr = (test_suppr - test_suppr.mean()) / (test_suppr.max() - test_suppr.min())

X_suppr , test_suppr, y_suppr, Id_test

In [ ]:
#VERIFICATION : NOMBRE DE DONNEES MANQUANTES
trainmanquantes=donnees_suppr()[0]
testmanquantes=donnees_suppr()[1]
sum(trainmanquantes.isnull().values.ravel()),sum(testmanquantes.isnull().values.ravel())

In [ ]:
#DONNEES INCOMPLETES -> MEAN

def donnees_mean():
# GENERATION ET NORMALISATION DES DONNEES
    train_mean = train.copy()
    y_mean=train_mean['Survived'].copy()
    Id_X = train_mean['PassengerId'].copy()
    X_mean=train_mean.drop(columns = ['Survived', 'PassengerId'])

    X_mean = (X_mean - X_mean.mean()) / (X_mean.max() - X_mean.min())

# MODIFICATION : NAN->MEAN
    colonnes_a_corriger = train_incompletes
    for col in colonnes_a_corriger:
        col_mean = X_mean[col].mean()
        X_mean[col]=X_mean[col].fillna(col_mean)



#PREPARATION DES DONNEES TEST
    test_mean = test.copy()
    
    Id_test = test_mean['PassengerId'].copy()
    test_mean=test_mean.drop(columns = 'PassengerId')
    
    test_mean=(test_mean - test_mean.mean()) / (test_mean.max() - test_mean.min())

# MODIFICATION DES DONNEES TEST : NAN->MEAN
    colonnes_a_corriger = test_incompletes
    for col in colonnes_a_corriger:
        col_mean = test_mean[col].mean()
        test_mean[col]=test_mean[col].fillna(col_mean)


    return X_mean, test_mean, y_mean, Id_test


In [ ]:
#VERIFICATION : NOMBRE DE DONNEES MANQUANTES
trainmanquantes=donnees_mean()[0]
testmanquantes=donnees_mean()[1]
sum(trainmanquantes.isnull().values.ravel()),sum(testmanquantes.isnull().values.ravel())

In [ ]:
#DONNEES INCOMPLETES -> INTREPOLATION

def donnees_interpolees():
# GENERATION ET NORMALISATION DES DONNEES
    train_interp = train.copy()
    y_interp=train_interp['Survived'].copy()
    Id_X = train_interp['PassengerId'].copy()
    X_interp=train_interp.drop(columns = ['Survived', 'PassengerId'])

    X_interp = (X_interp - X_interp.mean()) / (X_interp.max() - X_interp.min())
    
#PREPARATION DES DONNEES TEST
    test_interp = test.copy()
    
    Id_test = test_interp['PassengerId'].copy()
    test_interp=test_interp.drop(columns = 'PassengerId')
    
    test_interp = (test_interp - test_interp.mean()) / (test_interp.max() - test_interp.min())

# MODIFICATION : NAN->INTERPOLATION
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    imp = IterativeImputer(random_state=0)
    X_interp = imp.fit_transform(X_interp)
    test_interp = imp.fit_transform(test_interp)

    return X_interp, test_interp, y_interp, Id_test

In [ ]:
#VERIFICATION : NOMBRE DE DONNEES MANQUANTES
trainmanquantes=donnees_interpolees()[0]
testmanquantes=donnees_interpolees()[1]
np.count_nonzero(np.isnan(trainmanquantes)), np.count_nonzero(np.isnan(testmanquantes)), 

In [ ]:
#LISTE D'ALGORITHMES DE PREDICTION
from sklearn import linear_model
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB

Ridge = linear_model.RidgeClassifier()
LinReg = linear_model.LogisticRegression()
SDG = linear_model.SGDClassifier() #Stochastic Gradient Decent
PassiveAgressive = linear_model.PassiveAggressiveClassifier()
LDA = LinearDiscriminantAnalysis()
QDA = QuadraticDiscriminantAnalysis()
Tree = tree.DecisionTreeClassifier()

Mlpc_R_L = MLPClassifier(activation = 'relu', solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = lbfgs, activation = relu
Mlpc_R_S = MLPClassifier(activation = 'relu', solver='sgd', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = lbfgs, activation = sgd
Mlpc_R_A = MLPClassifier(activation = 'relu', solver='adam', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = lbfgs, activation = adam
Mlpc_T_L = MLPClassifier(activation = 'tanh', solver='lbfgs', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = tanh, activation = relu
Mlpc_T_S = MLPClassifier(activation = 'tanh', solver='sgd', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = tanh, activation = sgd
Mlpc_T_A = MLPClassifier(activation = 'tanh', solver='adam', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = tanh, activation = adam
Mlpc_L_L = MLPClassifier(activation = 'logistic', solver='lbfgs', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = logistic, activation = relu
Mlpc_L_S = MLPClassifier(activation = 'logistic', solver='sgd', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = logistic, activation = sgd
Mlpc_L_A = MLPClassifier(activation = 'logistic', solver='adam', max_iter=1000, alpha=1e-5, hidden_layer_sizes=(10, 2), random_state=1) # Neural Network solver = logistic, activation = adam

SVC = sk.svm.SVC()
Kneighbors = sk.neighbors.KNeighborsClassifier()
GaussProcess = GaussianProcessClassifier()
GaussNB = GaussianNB()
RandomForest = RandomForestClassifier()
AdaBoost = AdaBoostClassifier()

algorithme=[Ridge, LinReg, SDG, PassiveAgressive, LDA, QDA, Tree, Mlpc_R_L, Mlpc_R_S, Mlpc_R_A, Mlpc_T_L,Mlpc_T_S, Mlpc_T_A, Mlpc_L_L, Mlpc_L_S, Mlpc_L_A, SVC,Kneighbors, GaussProcess, RandomForest, AdaBoost]

In [ ]:
# TEST SVM - SVC
test_SVM_SVC = sk.svm.SVC()
test_SVM_SVC.fit(X_suppr, y_suppr)
test_SVM_SVC.predict(test_suppr)

In [ ]:
methode = [donnees_suppr(), donnees_mean(), donnees_interpolees()]
nom_methode = ['suppr','mean','interpolation']
nom_algo = ['Ridge', 'LinReg', 'SDG', 'PassiveAgressive', 'LDA', 'QDA', 'Tree', 'Mlpc_R_L', 'Mlpc_R_S', 'Mlpc_R_A', 'Mlpc_T_L', 'Mlpc_T_S', 'Mlpc_T_A', 'Mlpc_L_L', 'Mlpc_L_S', 'Mlpc_L_A', 'SVC', 'Kneighbors', 'GaussProcess', 'GaussNB', 'RandomForest', 'AdaBoost']
Sorties=[]

for k in range(len(methode)):
    [data_set, data_test, y, Id]=methode[k]
    
    for i in range(len(algorithme)):
        algo=algorithme[i]
        algo.fit(data_set, y)
        prediction = algo.predict(data_test)
        #Sorties.append([nom_methode[k] , nom_algo[i] , prediction])
        
        name  = nom_methode[k]+'_'+nom_algo[i]+'.csv'
        output = {'PassengerId' : Id, 'Survived' : prediction}
        pd.DataFrame(output).to_csv(name, index=False)


        
    


In [ ]:
Sorties[0][0]+'_'+Sorties[1][0]+'.csv'

In [ ]:
prevision = pd.read_csv('../kaggle/working/mean_GaussNB.csv')
prevision

In [ ]:
#
